# Importing Necessary Libraries for Data Preprocessing




In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
import plotly.express as px
from scipy.sparse import hstack
import re

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Add the Drive Path of the Sponsor Data
df = pd.read_csv('/content/drive/MyDrive/Agent_data.csv')

<ipython-input-4-20eac31f9b2a>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Agent_data.csv')


In [ ]:
df

,Id,f_name,l_name,website,email,lic_num,lic_num_state,otherlic,broker_lic_state,phone,brokername,address,description,team,service_area,jobtitle,specialities,languages,rating
0,1,Rommy,Shy,http://www.royaltyrealty.co/rommy-shy/,rshy@royaltyrealty.co|info@royaltyrealty.co|sy...,1874848,CA,1874848,Real Estate Sales,(818) 274-2655,Royalty Realty,"600 S. Spring St. Suite 105, Los Angeles,, CA,...",Real Estate Professional (11 years experience)...,NaN,"Commerce, CA|East Los Angeles, CA|Los Angeles ...",NaN,Buyer's Agent| Listing Agent| Relocation| Cons...,English| Spanish,4.9
1,2,Joseph,Polizzi,http://www.teampolizzi.com,joepolizzirealtor@gmail.com,8638279,NJ,NaN,NaN,(973) 828-0391,Keller Williams Realty Metropolitan,"333 Route 46 West, Mountain Lakes,, NJ, 07046",Your NJ Specialist (34 years experience) Speci...,NaN,"Boonton Township, NJ|Essex County, NJ|Hoboken,...",NaN,Buyer's Agent| Listing Agent| Relocation| Shor...,NaN,5.0
2,3,Amy,Carter,http://www.bedfordareahomes4sale.com,ccpa@marketleader.com|info@marketleader.com|dm...,NaN,NaN,0225 199040,Real Estate Sales,(540) 874-4113,Divine Fog Realty Company,"2203 GravesMill Rd Ste E, Forest,, VA, 24551",Realtor (9 years experience) Specialties: Buye...,NaN,"Goode, VA|Huddleston, VA|Roanoke, VA|Thaxton, VA",NaN,Buyer's Agent| Listing Agent| Staging,NaN,4.9
3,4,Susan,Tucker,"http://Susan Tucker,",NaN,7837325,NJ,7837325,Sales Associate,(908) 432-1388,Coldwell Banker Residential Brokerage,"7 Mt. Bethel Road, Warren,, NJ, 07069","Specialties: Buyer's Agent, Listing Agent Add ...",NaN,"Watchung, NJ",NaN,Buyer's Agent| Listing Agent,NaN,5.0
4,5,Kathlene,Holzhauer,https://kathlene.novapropertiesforsale.com/,consumerprivacy@boomtownroi.com|kathlene@teamd...,NaN,NaN,NaN,NaN,(703) 215-9832,Debbie Dogrul Associates - Long and Foster,"3918 Prosperity Ave #200, Fairfax,, VA, 22031",Realtor (11 years experience) Specialties: Lis...,NaN,"Fairfax, VA",Member of Debbie Dogrul Associates,Listing Agent| Relocation| Consulting| Landlor...,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499997,500181,Misty,Matthews,NaN,NaN,NaN,NaN,NaN,NaN,(336) 749-4791,Century 21 Hudspeth Properties,"968 N Bridge Street, Elkin,, NC, 28621","Specialties: Buyer's Agent, Listing Agent Acti...",NaN,"Elkin, NC",NaN,Buyer's Agent| Listing Agent,NaN,0.0
499998,500182,Hugh,Wallnutt,NaN,NaN,NaN,NaN,NaN,NaN,(719) 648-1591,"RE/MAX PROPERTIES, INC.","1424 Wood Ave, Colorado Springs,, CO, 80907","Specialties: Buyer's Agent, Listing Agent Acti...",NaN,"Colorado Springs, CO",NaN,Buyer's Agent| Listing Agent,NaN,0.0
499999,500183,Tyler,Reber,http://www.tylerereber.com,No Email Found,NaN,NaN,275374,Real Estate Sales,(252) 573-8601,ERA Strother Real Estate,"2505 Dalrymple Street, Sanford,, NC, 27332","Broker Associate Specialties: Buyer's Agent, L...",NaN,"Avon, NC|Corolla, NC|Hatteras, NC|Kill Devil H...",NaN,Buyer's Agent| Listing Agent| Relocation| Inve...,NaN,0.0
500000,500184,The,Trish Miller Group,http://WWW.TRISHMILLERAZ.COM,locations@hsmove.com,BR550092000,AZ,NaN,NaN,(928) 830-2586,Home Smart Fine Homes and Land,"140 N Montezuma , PRESCOTT,, AZ, 86301","Associate Broker, Real estate sales (17 years ...",NaN,"Chino Valley, AZ|Dewey, OK|Paulden, AZ|Prescot...",Lead of The Trish Miller Group,Buyer's Agent| Listing Agent| Short-Sale| Cert...,NaN,5.0


In [ ]:
# Summary Statistics
summary = df.describe()
summary

,rating
count,497195.000000
mean,2.406455
std,2.475687
min,0.000000
25%,0.000000
50%,0.000000
75%,5.000000
max,5.000000


In [ ]:
# Number of Duplicates
duplicate_count = df.duplicated().sum()
duplicate_count

0

In [ ]:
# Null Values
null_counts = df.isnull().sum()
null_counts

,0
Id,0
f_name,23
l_name,2223
website,173251
email,173344
lic_num,355555
lic_num_state,355556
otherlic,289120
broker_lic_state,289667
phone,47057


In [ ]:
# Percentage of Null Values
null_percentages = (df.isnull().sum() / len(df)) * 100
null_percentages

,0
Id,0.000000
f_name,0.004600
l_name,0.444598
website,34.650061
email,34.668661
lic_num,71.110716
lic_num_state,71.110916
otherlic,57.823769
broker_lic_state,57.933168
phone,9.411362


# Data Cleaning and Preprocessing

In [ ]:
# Extract the language names and insert them into a new column
df['languages'] = df['languages'].str.findall(r'[A-Za-z]+').apply(lambda x: ', '.join(x) if isinstance(x, list) else 'English')

In [ ]:
# Handling NaN values and extracting specialities separated by "|"
df['specialities'] = df['specialities'].fillna('None').str.split('|').apply(
    lambda x: ', '.join([item.strip() for item in x if isinstance(item, str)])
)

In [ ]:
# Extracting only 'Member' and 'Lead' from the 'Description' column
df['jobtitle'] = df['jobtitle'].fillna('').apply(
    lambda x: 'Member' if 'Member' in x else ('Lead' if 'Lead' in x else np.nan)
)

In [ ]:
# Filling 'jobtitle' based on the condition: 'Individual' if 'brokername' is NaN, 'Member' if 'brokername' has data
df['jobtitle'] = df.apply(
    lambda row: 'Individual' if pd.isna(row['jobtitle']) and pd.isna(row['brokername'])
    else ('Member' if pd.isna(row['jobtitle']) and not pd.isna(row['brokername']) else row['jobtitle']),
    axis=1
)

In [ ]:
# Vectorized function to separate city and state using regular expressions
def vectorized_separation(data):
    # Extract city names and states using regex
    city_extracted = data.str.findall(r'([^,]+), [A-Z]{2}')
    state_extracted = data.str.findall(r', ([A-Z]{2})')

    # Convert lists to strings, remove duplicates for states
    cities = city_extracted.apply(lambda x: ', '.join(set(x)) if isinstance(x, list) else 'Not Available')
    states = state_extracted.apply(lambda x: ', '.join(sorted(set(x))) if isinstance(x, list) else 'Not Available')

    return pd.DataFrame({'City': cities, 'State': states})

# Apply the vectorized function to the 'service_area' column
df[['City', 'State']] = vectorized_separation(df['service_area'])


In [ ]:
df['City'] = df['City'].str.replace('|', '', regex=False).str.strip()

In [ ]:
# Updated function to extract job title, experience (as integer), and specialties only
def extract_profile_info_integer_experience(text):
    # Ensure the input is a string; if not, return empty strings
    if not isinstance(text, str):
        return pd.Series(['', '', ''])

    # Initialize variables
    job_title = specialties = ''
    experience = 0

    # Extract job title and experience using regex
    title_exp_match = re.search(r'([A-Za-z\s]+)\s\((\d+) years? experience\)', text)
    if title_exp_match:
        job_title = title_exp_match.group(1).strip()
        experience = int(title_exp_match.group(2))  # Convert to integer

    # Extract specialties using regex
    specialties_match = re.search(r'Specialties:\s*([^\n]+?)(?=Add personality to your profile!)', text)
    if specialties_match:
        specialties = specialties_match.group(1).strip()

    return pd.Series([job_title, experience, specialties])

In [ ]:
# Apply the optimized function to the 'description' column (Takes around 10-15 mins)
df[['Job Title', 'Experience', 'Specialities_1']] = df['description'].apply(extract_profile_info_integer_experience)

In [ ]:
# List of All the columns with updated columns
columns_list = df.columns.tolist()
print(columns_list)

['Id', 'f_name', 'l_name', 'website', 'email', 'lic_num', 'lic_num_state', 'otherlic', 'broker_lic_state', 'phone', 'brokername', 'address', 'description', 'team', 'service_area', 'jobtitle', 'specialities', 'languages', 'rating', 'City', 'State', 'Job Title', 'Experience', 'Specialities_1']


In [ ]:
# Handle other columns not extracted from description
df['f_name'] = df['f_name'].fillna('')
df['l_name'] = df['l_name'].fillna('')
df['lic_num'] = df['lic_num'].fillna('Not Available')
df['lic_num_state'] = df['lic_num_state'].fillna('Not Available')
df['otherlic'] = df['otherlic'].fillna('Not Available')
df['broker_lic_state'] = df['broker_lic_state'].fillna('Not Available')
df['phone'] = df['phone'].fillna('Not Available')
df['brokername'] = df['brokername'].fillna('Independent')
df['address'] = df['address'].fillna('Not Available')
df['email'] = df['email'].fillna('Not Available')
df['website'] = df['website'].fillna('Not Available')
df['rating'] = df['rating'].fillna(df['rating'].median())
# Replace empty strings with NaN
df['Experience'].replace(0, np.nan, inplace=True)
df['Experience'] = pd.to_numeric(df['Experience'], errors='coerce')
median_value = df['Experience'].median()
df['Experience'].fillna(median_value, inplace=True)
df['Experience'] = df['Experience'].fillna(df['Experience'].median())
df.drop(columns=['team'], inplace=True)

<ipython-input-21-f3907f9c2557>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Experience'].replace(0, np.nan, inplace=True)
<ipython-input-21-f3907f9c2557>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [ ]:
# Replace empty strings with NaN
df['Experience'].replace(0, np.nan, inplace=True)

# Convert 'Experience' column to numeric type, coercing errors to NaN
df['Experience'] = pd.to_numeric(df['Experience'], errors='coerce')

# Calculate median value, ignoring NaN values
median_value = df['Experience'].median()

# Fill NaN values with the median
df['Experience'].fillna(median_value, inplace=True)
df['Experience'] = df['Experience'].fillna(df['Experience'].median())

<ipython-input-22-1d917a839c5e>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Experience'].replace(0, np.nan, inplace=True)
<ipython-input-22-1d917a839c5e>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [ ]:
null_counts = df.isnull().sum()
null_counts

,0
Id,0
f_name,0
l_name,0
website,0
email,0
lic_num,0
lic_num_state,0
otherlic,0
broker_lic_state,0
phone,0


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/cleanDF_loc.csv')
df1

,Unnamed: 0,Id,f_name,l_name,description,specialities,rating,cleaned_description,service_area_clean,latitude,longitude
0,0,1,Rommy,Shy,Real Estate Professional (11 years experience)...,Buyer's Agent| Listing Agent| Relocation| Cons...,4.9,"['real', 'estate', 'professional', '11', 'year...","['Commerce, CA', 'East Los Angeles, CA', 'Los ...",33.990686,-118.143914
1,1,2,Joseph,Polizzi,Your NJ Specialist (34 years experience) Speci...,Buyer's Agent| Listing Agent| Relocation| Shor...,5.0,"['nj', 'specialist', '34', 'years', 'experienc...","['Boonton Township, NJ', 'Essex County, NJ', '...",NaN,NaN
2,2,3,Amy,Carter,Realtor (9 years experience) Specialties: Buye...,Buyer's Agent| Listing Agent| Staging,4.9,"['realtor', '9', 'years', 'experience', 'speci...","['Goode, VA', 'Huddleston, VA', 'Roanoke, VA',...",NaN,NaN
3,3,4,Susan,Tucker,"Specialties: Buyer's Agent, Listing Agent Add ...",Buyer's Agent| Listing Agent,5.0,"['specialties', 'buyer', 'agent', 'listing', '...","['Watchung, NJ ']",40.637880,-74.450986
4,4,5,Kathlene,Holzhauer,Realtor (11 years experience) Specialties: Lis...,Listing Agent| Relocation| Consulting| Landlor...,5.0,"['realtor', '11', 'years', 'experience', 'spec...","['Fairfax, VA ']",38.815636,-77.283685
...,...,...,...,...,...,...,...,...,...,...,...
499993,499993,500181,Misty,Matthews,"Specialties: Buyer's Agent, Listing Agent Acti...",Buyer's Agent| Listing Agent,0.0,"['specialties', 'buyer', 'agent', 'listing', '...","['Elkin, NC ']",NaN,NaN
499994,499994,500182,Hugh,Wallnutt,"Specialties: Buyer's Agent, Listing Agent Acti...",Buyer's Agent| Listing Agent,0.0,"['specialties', 'buyer', 'agent', 'listing', '...","['Colorado Springs, CO ']",NaN,NaN
499995,499995,500183,Tyler,Reber,"Broker Associate Specialties: Buyer's Agent, L...",Buyer's Agent| Listing Agent| Relocation| Inve...,0.0,"['broker', 'associate', 'specialties', 'buyer'...","['Avon, NC', 'Corolla, NC', 'Hatteras, NC', 'K...",NaN,NaN
499996,499996,500184,The,Trish Miller Group,"Associate Broker, Real estate sales (17 years ...",Buyer's Agent| Listing Agent| Short-Sale| Cert...,5.0,"['associate', 'broker', 'real', 'estate', 'sal...","['Chino Valley, AZ', 'Dewey, OK', 'Paulden, AZ...",NaN,NaN


In [ ]:
final_df = pd.merge(df, df1[['Id', 'latitude', 'longitude']], on='Id', how='left', suffixes=('', '_df1'))

In [ ]:
final_df

,Id,f_name,l_name,website,email,lic_num,lic_num_state,otherlic,broker_lic_state,phone,...,specialities,languages,rating,City,State,Job Title,Experience,Specialities_1,latitude,longitude
0,1,Rommy,Shy,http://www.royaltyrealty.co/rommy-shy/,rshy@royaltyrealty.co|info@royaltyrealty.co|sy...,1874848,CA,1874848,Real Estate Sales,(818) 274-2655,...,"Buyer's Agent, Listing Agent, Relocation, Cons...","English, Spanish",4.9,"Los Angeles County, Los Angeles, East Los Ange...",CA,Real Estate Professional,11.0,"Buyer's Agent, Listing Agent, Relocation, Cons...",33.990686,-118.143914
1,2,Joseph,Polizzi,http://www.teampolizzi.com,joepolizzirealtor@gmail.com,8638279,NJ,Not Available,Not Available,(973) 828-0391,...,"Buyer's Agent, Listing Agent, Relocation, Shor...",English,5.0,"Essex County, Hoboken, Monmouth County, Union ...",NJ,Your NJ Specialist,34.0,"Buyer's Agent, Listing Agent, Relocation, Shor...",NaN,NaN
2,3,Amy,Carter,http://www.bedfordareahomes4sale.com,ccpa@marketleader.com|info@marketleader.com|dm...,Not Available,Not Available,0225 199040,Real Estate Sales,(540) 874-4113,...,"Buyer's Agent, Listing Agent, Staging",English,4.9,"Roanoke, Goode, Huddleston, Thaxton",VA,Realtor,9.0,"Buyer's Agent, Listing Agent, Staging",NaN,NaN
3,4,Susan,Tucker,"http://Susan Tucker,",Not Available,7837325,NJ,7837325,Sales Associate,(908) 432-1388,...,"Buyer's Agent, Listing Agent",English,5.0,Watchung,NJ,,16.0,"Buyer's Agent, Listing Agent",40.637880,-74.450986
4,5,Kathlene,Holzhauer,https://kathlene.novapropertiesforsale.com/,consumerprivacy@boomtownroi.com|kathlene@teamd...,Not Available,Not Available,Not Available,Not Available,(703) 215-9832,...,"Listing Agent, Relocation, Consulting, Landlor...",English,5.0,Fairfax,VA,Realtor,11.0,"Listing Agent, Relocation, Consulting, Landlor...",38.815636,-77.283685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499997,500181,Misty,Matthews,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,(336) 749-4791,...,"Buyer's Agent, Listing Agent",English,0.0,Elkin,NC,,16.0,,NaN,NaN
499998,500182,Hugh,Wallnutt,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,(719) 648-1591,...,"Buyer's Agent, Listing Agent",English,0.0,Colorado Springs,CO,,16.0,,NaN,NaN
499999,500183,Tyler,Reber,http://www.tylerereber.com,No Email Found,Not Available,Not Available,275374,Real Estate Sales,(252) 573-8601,...,"Buyer's Agent, Listing Agent, Relocation, Inve...",English,0.0,"Avon, Kill Devil Hills, Hatteras, Manteo, Sout...",NC,,16.0,"Buyer's Agent, Listing Agent, Relocation, Inve...",NaN,NaN
500000,500184,The,Trish Miller Group,http://WWW.TRISHMILLERAZ.COM,locations@hsmove.com,BR550092000,AZ,Not Available,Not Available,(928) 830-2586,...,"Buyer's Agent, Listing Agent, Short-Sale, Cert...",English,5.0,"Prescott Valley, Paulden, Chino Valley, Dewey,...","AZ, OK",Real estate sales,17.0,"Buyer's Agent, Listing Agent, Short-Sale, Cert...",NaN,NaN


In [ ]:
# Save the DataFrame to a CSV file
csv_file_path = 'Final_df.csv'
final_df.to_csv(csv_file_path, index=False)

## Preparing for Encoding (Languages and Specialities)



In [ ]:
import pandas as pd

In [ ]:
final_df = pd.read_csv('~/Downloads/Final_df.csv', low_memory=False)

In [ ]:
final_df

,Id,f_name,l_name,website,email,lic_num,lic_num_state,otherlic,broker_lic_state,phone,...,specialities,languages,rating,City,State,Job Title,Experience,Specialities_1,latitude,longitude
0,1,Rommy,Shy,http://www.royaltyrealty.co/rommy-shy/,rshy@royaltyrealty.co|info@royaltyrealty.co|sy...,1874848,CA,1874848,Real Estate Sales,(818) 274-2655,...,"Buyer's Agent, Listing Agent, Relocation, Cons...","English, Spanish",4.9,"Los Angeles County, Los Angeles, East Los Ange...",CA,Real Estate Professional,11.0,"Buyer's Agent, Listing Agent, Relocation, Cons...",33.990686,-118.143914
1,2,Joseph,Polizzi,http://www.teampolizzi.com,joepolizzirealtor@gmail.com,8638279,NJ,Not Available,Not Available,(973) 828-0391,...,"Buyer's Agent, Listing Agent, Relocation, Shor...",English,5.0,"Essex County, Hoboken, Monmouth County, Union ...",NJ,Your NJ Specialist,34.0,"Buyer's Agent, Listing Agent, Relocation, Shor...",NaN,NaN
2,3,Amy,Carter,http://www.bedfordareahomes4sale.com,ccpa@marketleader.com|info@marketleader.com|dm...,Not Available,Not Available,0225 199040,Real Estate Sales,(540) 874-4113,...,"Buyer's Agent, Listing Agent, Staging",English,4.9,"Roanoke, Goode, Huddleston, Thaxton",VA,Realtor,9.0,"Buyer's Agent, Listing Agent, Staging",NaN,NaN
3,4,Susan,Tucker,"http://Susan Tucker,",Not Available,7837325,NJ,7837325,Sales Associate,(908) 432-1388,...,"Buyer's Agent, Listing Agent",English,5.0,Watchung,NJ,NaN,16.0,"Buyer's Agent, Listing Agent",40.637880,-74.450986
4,5,Kathlene,Holzhauer,https://kathlene.novapropertiesforsale.com/,consumerprivacy@boomtownroi.com|kathlene@teamd...,Not Available,Not Available,Not Available,Not Available,(703) 215-9832,...,"Listing Agent, Relocation, Consulting, Landlor...",English,5.0,Fairfax,VA,Realtor,11.0,"Listing Agent, Relocation, Consulting, Landlor...",38.815636,-77.283685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499997,500181,Misty,Matthews,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,(336) 749-4791,...,"Buyer's Agent, Listing Agent",English,0.0,Elkin,NC,NaN,16.0,NaN,NaN,NaN
499998,500182,Hugh,Wallnutt,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,(719) 648-1591,...,"Buyer's Agent, Listing Agent",English,0.0,Colorado Springs,CO,NaN,16.0,NaN,NaN,NaN
499999,500183,Tyler,Reber,http://www.tylerereber.com,No Email Found,Not Available,Not Available,275374,Real Estate Sales,(252) 573-8601,...,"Buyer's Agent, Listing Agent, Relocation, Inve...",English,0.0,"Avon, Kill Devil Hills, Hatteras, Manteo, Sout...",NC,NaN,16.0,"Buyer's Agent, Listing Agent, Relocation, Inve...",NaN,NaN
500000,500184,The,Trish Miller Group,http://WWW.TRISHMILLERAZ.COM,locations@hsmove.com,BR550092000,AZ,Not Available,Not Available,(928) 830-2586,...,"Buyer's Agent, Listing Agent, Short-Sale, Cert...",English,5.0,"Prescott Valley, Paulden, Chino Valley, Dewey,...","AZ, OK",Real estate sales,17.0,"Buyer's Agent, Listing Agent, Short-Sale, Cert...",NaN,NaN


In [ ]:
# Display column name, datatype, null values, and number of duplicates
columns_info = pd.DataFrame({
    'Column Name': final_df.columns,
    'Data Type': final_df.dtypes,
    'Null Values': final_df.isnull().sum(),
    'Duplicate Values': final_df.duplicated().sum() if final_df.duplicated().any() else [0] * len(final_df.columns)
})

print(columns_info)

                       Column Name Data Type  Null Values  Duplicate Values
Id                              Id    object            0                 0
f_name                      f_name    object           23                 0
l_name                      l_name    object         2223                 0
website                    website    object            0                 0
email                        email    object            0                 0
lic_num                    lic_num    object            0                 0
lic_num_state        lic_num_state    object            0                 0
otherlic                  otherlic    object            0                 0
broker_lic_state  broker_lic_state    object            0                 0
phone                        phone    object            0                 0
brokername              brokername    object            0                 0
address                    address    object            0                 0
description 

## Encoding ("Languages" and "Specialities")

In [ ]:
import pandas as pd
from collections import Counter

# Initialize counters for languages and specialities
language_counter = Counter()
speciality_counter = Counter()

# Define chunk size
chunk_size = 500000

# Process the DataFrame in chunks
for chunk_number, start_idx in enumerate(range(0, len(final_df), chunk_size), start=1):
    print(f"Processing chunk {chunk_number}...")

    # Extract the chunk
    chunk = final_df.iloc[start_idx:start_idx + chunk_size].copy()

    # Split 'languages' and 'specialities' columns into lists
    chunk['languages'] = chunk['languages'].fillna('').str.split(', ')
    chunk['specialities'] = chunk['specialities'].fillna('').str.split(', ')

    # Count frequencies in the current chunk
    for langs in chunk['languages']:
        if isinstance(langs, list):
            language_counter.update(langs)

    for specs in chunk['specialities']:
        if isinstance(specs, list):
            speciality_counter.update(specs)

# Identify top 20 languages and specialities
top_20_languages = [lang for lang, _ in language_counter.most_common(20)]
top_20_specialities = [spec for spec, _ in speciality_counter.most_common(20)]

# Remove overlap between top languages and specialities
top_20_languages = [lang for lang in top_20_languages if lang not in top_20_specialities]
top_20_specialities = [spec for spec in top_20_specialities if spec not in top_20_languages]

# Combine top languages and specialities
top_columns = top_20_languages + top_20_specialities

print("Top 20 Languages:", top_20_languages)
print("Top 20 Specialities:", top_20_specialities)
print("Total Top Columns:", top_columns)


Processing chunk 1...
Processing chunk 2...
Top 20 Languages: ['English', 'Spanish', 'French', 'Mandarin', 'Russian', 'Portuguese', 'German', 'Italian', 'Hindi', 'Cantonese', 'Arabic', 'Farsi', 'Korean', 'Vietnamese', 'Hebrew', 'Polish', 'Tagalog', 'Filipino', 'Greek', 'Japanese']
Top 20 Specialities: ['Listing Agent', "Buyer's Agent", 'Relocation', 'Property Management', 'Consulting', 'Foreclosure', 'Short-Sale', 'None', 'Landlord', 'Staging', 'Commercial R.E.', 'Notary', 'Appraisal', 'Vacation / Short-term Rental', 'Moving', 'General Contracting', 'Insurance', 'New Construction', 'Legal', 'General Inspection']
Total Top Columns: ['English', 'Spanish', 'French', 'Mandarin', 'Russian', 'Portuguese', 'German', 'Italian', 'Hindi', 'Cantonese', 'Arabic', 'Farsi', 'Korean', 'Vietnamese', 'Hebrew', 'Polish', 'Tagalog', 'Filipino', 'Greek', 'Japanese', 'Listing Agent', "Buyer's Agent", 'Relocation', 'Property Management', 'Consulting', 'Foreclosure', 'Short-Sale', 'None', 'Landlord', 'Stagin

## Heavy RAM Usage (So sending the data in batch and then later merging them)

In [ ]:
# Define batch number and batch size
batch_number = 1  # Change this from 1 to 5 as you process each batch
batch_size = 100000  # Size of each batch

# Calculate start and end indices for the current batch
start_idx = (batch_number - 1) * batch_size
end_idx = batch_number * batch_size if batch_number * batch_size < len(final_df) else len(final_df)

# Select the current batch
batch = final_df.iloc[start_idx:end_idx].copy()

# Prepare 'languages' and 'specialities' columns
batch['languages'] = batch['languages'].fillna('').str.split(', ')
batch['specialities'] = batch['specialities'].fillna('').str.split(', ')

# Encode only the top columns
for column in top_columns:
    if column in top_20_languages:
        batch[column] = batch['languages'].apply(lambda x: 1 if isinstance(x, list) and column in x else 0)
    elif column in top_20_specialities:
        batch[column] = batch['specialities'].apply(lambda x: 1 if isinstance(x, list) and column in x else 0)

# Retain relevant columns
columns_to_keep = ['Id', 'lic_num', 'lic_num_state', 'latitude', 'longitude'] + top_columns
batch_filtered = batch[columns_to_keep]

# Save the processed batch
output_file = f'~/Downloads/Encoded_Batch_{batch_number}.csv'
batch_filtered.to_csv(output_file, index=False)
print(f"Batch {batch_number} processed and saved to {output_file}")


Batch 1 processed and saved to ~/Downloads/Encoded_Batch_1.csv


In [ ]:
# Combine all processed batches
batch_files = [
    '~/Downloads/Encoded_Batch_1.csv',
    '~/Downloads/Encoded_Batch_2.csv',
    '~/Downloads/Encoded_Batch_3.csv',
    '~/Downloads/Encoded_Batch_4.csv',
    '~/Downloads/Encoded_Batch_5.csv',
]

combined_df = pd.concat([pd.read_csv(file) for file in batch_files], ignore_index=True)
combined_df.to_csv('~/Downloads/Final_Encoded_DF.csv', index=False)
print("All batches combined and saved to Final_Encoded_DF.csv")

/var/folders/rd/5t2vy15d437ffncxbx1d4x2h0000gn/T/ipykernel_65467/4100457347.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.concat([pd.read_csv(file) for file in batch_files], ignore_index=True)
/var/folders/rd/5t2vy15d437ffncxbx1d4x2h0000gn/T/ipykernel_65467/4100457347.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.concat([pd.read_csv(file) for file in batch_files], ignore_index=True)


All batches combined and saved to Final_Encoded_DF.csv


In [ ]:
import pandas as pd

# Specify the input CSV file and output Excel file paths
input_csv_file = '~/Downloads/Final_Encoded_DF.csv'  # Replace with your CSV file path
output_excel_file = '~/Downloads/Final_Encoded_DF.xlsx'  # Replace with your desired XLSX file path

# Read the CSV file
df = pd.read_csv(input_csv_file)

# Save the DataFrame to an Excel file
df.to_excel(output_excel_file, index=False)

print(f"File converted and saved as {output_excel_file}")


/var/folders/rd/5t2vy15d437ffncxbx1d4x2h0000gn/T/ipykernel_65467/2032997237.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_csv_file)


File converted and saved as ~/Downloads/Final_Encoded_DF.xlsx


## Preparation for Recommendation System

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
import IPython.display as ipd

In [ ]:
agent_df = pd.read_csv('~/Downloads/Final_df.csv', low_memory=False)
review_df = pd.read_csv('~/Downloads/Agentreview_cleaned.csv', low_memory=False)
sales_df = pd.read_csv('~/Downloads/PastSalesAnalysis.csv', low_memory=False)

In [ ]:
agent_df

,Id,f_name,l_name,website,email,lic_num,lic_num_state,otherlic,broker_lic_state,phone,...,specialities,languages,rating,City,State,Job Title,Experience,Specialities_1,latitude,longitude
0,1,Rommy,Shy,http://www.royaltyrealty.co/rommy-shy/,rshy@royaltyrealty.co|info@royaltyrealty.co|sy...,1874848,CA,1874848,Real Estate Sales,(818) 274-2655,...,"Buyer's Agent, Listing Agent, Relocation, Cons...","English, Spanish",4.9,"Los Angeles County, Los Angeles, East Los Ange...",CA,Real Estate Professional,11.0,"Buyer's Agent, Listing Agent, Relocation, Cons...",33.990686,-118.143914
1,2,Joseph,Polizzi,http://www.teampolizzi.com,joepolizzirealtor@gmail.com,8638279,NJ,Not Available,Not Available,(973) 828-0391,...,"Buyer's Agent, Listing Agent, Relocation, Shor...",English,5.0,"Essex County, Hoboken, Monmouth County, Union ...",NJ,Your NJ Specialist,34.0,"Buyer's Agent, Listing Agent, Relocation, Shor...",NaN,NaN
2,3,Amy,Carter,http://www.bedfordareahomes4sale.com,ccpa@marketleader.com|info@marketleader.com|dm...,Not Available,Not Available,0225 199040,Real Estate Sales,(540) 874-4113,...,"Buyer's Agent, Listing Agent, Staging",English,4.9,"Roanoke, Goode, Huddleston, Thaxton",VA,Realtor,9.0,"Buyer's Agent, Listing Agent, Staging",NaN,NaN
3,4,Susan,Tucker,"http://Susan Tucker,",Not Available,7837325,NJ,7837325,Sales Associate,(908) 432-1388,...,"Buyer's Agent, Listing Agent",English,5.0,Watchung,NJ,NaN,16.0,"Buyer's Agent, Listing Agent",40.637880,-74.450986
4,5,Kathlene,Holzhauer,https://kathlene.novapropertiesforsale.com/,consumerprivacy@boomtownroi.com|kathlene@teamd...,Not Available,Not Available,Not Available,Not Available,(703) 215-9832,...,"Listing Agent, Relocation, Consulting, Landlor...",English,5.0,Fairfax,VA,Realtor,11.0,"Listing Agent, Relocation, Consulting, Landlor...",38.815636,-77.283685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499997,500181,Misty,Matthews,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,(336) 749-4791,...,"Buyer's Agent, Listing Agent",English,0.0,Elkin,NC,NaN,16.0,NaN,NaN,NaN
499998,500182,Hugh,Wallnutt,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,(719) 648-1591,...,"Buyer's Agent, Listing Agent",English,0.0,Colorado Springs,CO,NaN,16.0,NaN,NaN,NaN
499999,500183,Tyler,Reber,http://www.tylerereber.com,No Email Found,Not Available,Not Available,275374,Real Estate Sales,(252) 573-8601,...,"Buyer's Agent, Listing Agent, Relocation, Inve...",English,0.0,"Avon, Kill Devil Hills, Hatteras, Manteo, Sout...",NC,NaN,16.0,"Buyer's Agent, Listing Agent, Relocation, Inve...",NaN,NaN
500000,500184,The,Trish Miller Group,http://WWW.TRISHMILLERAZ.COM,locations@hsmove.com,BR550092000,AZ,Not Available,Not Available,(928) 830-2586,...,"Buyer's Agent, Listing Agent, Short-Sale, Cert...",English,5.0,"Prescott Valley, Paulden, Chino Valley, Dewey,...","AZ, OK",Real estate sales,17.0,"Buyer's Agent, Listing Agent, Short-Sale, Cert...",NaN,NaN


In [ ]:
print(list(agent_df.columns))

['Id', 'f_name', 'l_name', 'website', 'email', 'lic_num', 'lic_num_state', 'otherlic', 'broker_lic_state', 'phone', 'brokername', 'address', 'description', 'service_area', 'jobtitle', 'specialities', 'languages', 'rating', 'City', 'State', 'Job Title', 'Experience', 'Specialities_1', 'latitude', 'longitude']


# Recommendation System

In [ ]:
# Fill missing values (example: using the mean)
agent_df['Experience'] = agent_df['Experience'].fillna(agent_df['Experience'].mean())
agent_df['Rating'] = agent_df['rating'].fillna(agent_df['rating'].mean())

In [ ]:
# Step 1: Calculate a combined "fit score" based on Experience and Rating
# You can adjust the weights based on your preferences
agent_df['fit_score'] = 0.5 * agent_df['Experience'] + 0.5 * agent_df['rating']

In [ ]:
# Step 2: Ask the user for input to filter agents by location
location_input = input("Enter the location (zipcode, address, or city) you are looking for: ")

Enter the location (zipcode, address, or city) you are looking for:  san jose


In [ ]:
# Step 3: Filter agents based on the location input
# Split the location input into keywords (e.g., if user types "Los Angeles", keywords will be ["Los", "Angeles"])
location_keywords = location_input.split()

# Ensure that 'service_area' is a string (some values might be lists or NaN)
agent_df['service_area'] = agent_df['service_area'].fillna('').astype(str)

In [ ]:
# Step 5: Filter the agents based on the service area matching any of the location keywords
data_filtered = agent_df[agent_df['service_area'].apply(
    lambda x: any(keyword.lower() in x.lower() for keyword in location_keywords))]

In [ ]:
# Step 6: Sort the filtered agents by the fit score to find the best ones
best_agents = data_filtered.sort_values(by='fit_score', ascending=False).head(10)

# Step 7: Display the best agents found based on location
print("Top 10 agents based on your location search:")
print(best_agents[['f_name', 'l_name', 'service_area', 'fit_score', 'Experience', 'Rating', 'phone', 'email']])

Top 10 agents based on your location search:
           f_name                               l_name  \
452725       Lora                               DeBord   
55171       Patti                              Chaffee   
238912        The                        Ohlmeyer Team   
419613  Byzantium                            Brokerage   
381514   McCarthy                   Transfer & Storage   
165990     Heyler                          Realty Team   
163341        KBB                             Realtors   
13613        Cass                          Real Estate   
135615     McGraw                             Realtors   
96795         VIP  Group-Saundra, Chad, Deanna, Kellie   

                                             service_area  fit_score  \
452725                                   San Antonio, TX       60.00   
55171   Brentwood, CA|Culver City, CA|Los Angeles, CA|...      60.00   
238912  Alameda County, CA|Alamo, CA|Antioch, CA|Bay P...      57.00   
419613  Cow Hollow, San Fran

In [ ]:
# Display the best agents in the specified area
import IPython.display as ipd

# Visual representation of the top 20 agents to contact
import folium
from folium.plugins import MarkerCluster

# Filter the top 20 agents by fit score for visualization
best_20_agents = data_filtered.sort_values(by='fit_score', ascending=False).head(20)

# Create a map centered around a general location (e.g., United States)
agent_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)
marker_cluster = MarkerCluster().add_to(agent_map)

# Add agent locations to the map (use approximate latitude and longitude if actual data is not available)
for _, agent in best_20_agents.iterrows():
    # Placeholder: Assuming we have some approximate latitude and longitude based on service area
    # You can use geocoding services to get exact lat/lon based on addresses
    latitude = np.random.uniform(25, 50)  # Replace with actual latitude if available
    longitude = np.random.uniform(-125, -67)  # Replace with actual longitude if available

    folium.Marker(
        location=[latitude, longitude],
        popup=(
            f"<b>Name:</b> {agent['f_name']} {agent['l_name']}<br>"
            f"<b>Specialities:</b> {agent['specialities']}<br>"
            f"<b>Service Area:</b> {agent['service_area']}<br>"
            f"<b>Fit Score:</b> {agent['fit_score']}"
        ),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

# Save and display the map
map_file_path = "agents_map.html"
agent_map.save(map_file_path)
try:
    ipd.display(ipd.IFrame(src=map_file_path, width='100%', height='600px'))
except Exception as e:
    print(f"Unable to display the map: {e}")